LeNet is the first convolutional neural network. This note book reproduces the LeNet using Fashion-MNIST dataset.

In [1]:
%matplotlib inline
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
import torch.optim as optim
import time

batch_size = 256
num_epochs = 20
transform = transforms.Compose([transforms.ToTensor()]) 

mnist_trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

LeNet uses two convolutional layers to extract the input features, and three fully connected layers are used as classifier.

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.sigmoid = nn.Sigmoid()
        self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        # Feature extraction using to convolutional layers.
        x = self.avgpool(self.sigmoid(self.conv1(x)))
        x = self.avgpool(self.sigmoid(self.conv2(x)))
        #reshape the tensor to 1-d to fit the FC layer input
        x = x.view(x.shape[0], -1)
        # Classifier using three fully connected layers.
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.normal_(0.0, 0.01)
    elif classname.find('Linear') != -1:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.normal_(0.0, 0.01)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.01)
        m.bias.data.fill_(0)

def evaluate_accuracy(data_iter, net):
    """Evaluate accuracy of a model on the given data set."""
    acc_sum,n = 0,0
    for (imgs, labels) in data_iter:
        # send data to the GPU if cuda is availabel
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()
        net.eval()
        with torch.no_grad():
            labels = labels.long()
            acc_sum += torch.sum((torch.argmax(net(imgs), dim=1) == labels)).float()
            n += labels.shape[0]
    return acc_sum.item()/n

In [3]:
# Loading training set and test set using DataLoader.
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=batch_size,
    shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=batch_size,
    shuffle=True, num_workers=0)

if torch.cuda.is_available():
    print('Training using GPU.')
    net = LeNet().cuda()
else:
    print('Training using CPU.')
    net = LeNet()

#Initialize network parameters.
net.apply(weights_init)

#Loss function
if torch.cuda.is_available():
    loss = nn.CrossEntropyLoss().cuda()
else:
    loss = nn.CrossEntropyLoss()

# Train using SGD optimizer 
lr= 0.3 # This learning rate was not fine-tuned.
opt_n = optim.SGD(net.parameters(), lr=lr)

# Training stage
for epoch in range(1, num_epochs+1):
    train_loader_iter = iter(train_loader)
    train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
    
    for (imgs, labels) in train_loader_iter:
        net.train()
        opt_n.zero_grad()
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()
        # Label prediction from LeNet
        y_hat = net(imgs)
        l = loss(y_hat, labels)
        # Backprobagation
        l.backward()
        opt_n.step()

        # Calculate tarining error
        with torch.no_grad():
            labels = labels.long()
            train_l_sum += l.item()
            train_acc_sum += (torch.sum(torch.argmax(y_hat, dim=1) == labels)).float().item()
            n += labels.shape[0]
    # calculate testing error every epoch.
    test_acc = evaluate_accuracy(iter(test_loader), net)
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
          % (epoch, train_l_sum/n, train_acc_sum/n, test_acc,
            time.time() - start))


            



Training using GPU.
epoch 1, loss 0.0091, train acc 0.099, test acc 0.100, time 4.4 sec
epoch 2, loss 0.0090, train acc 0.110, test acc 0.100, time 4.4 sec
epoch 3, loss 0.0066, train acc 0.410, test acc 0.568, time 4.4 sec
epoch 4, loss 0.0040, train acc 0.608, test acc 0.608, time 4.4 sec
epoch 5, loss 0.0034, train acc 0.673, test acc 0.701, time 4.5 sec
epoch 6, loss 0.0031, train acc 0.698, test acc 0.676, time 4.5 sec
epoch 7, loss 0.0029, train acc 0.720, test acc 0.692, time 4.5 sec
epoch 8, loss 0.0027, train acc 0.735, test acc 0.714, time 4.5 sec
epoch 9, loss 0.0026, train acc 0.744, test acc 0.738, time 4.5 sec
epoch 10, loss 0.0025, train acc 0.751, test acc 0.683, time 4.4 sec
epoch 11, loss 0.0024, train acc 0.762, test acc 0.762, time 4.5 sec
epoch 12, loss 0.0023, train acc 0.770, test acc 0.754, time 4.5 sec
epoch 13, loss 0.0023, train acc 0.775, test acc 0.766, time 4.5 sec
epoch 14, loss 0.0022, train acc 0.784, test acc 0.764, time 4.6 sec
epoch 15, loss 0.0021, 